[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jburgy/blog/blob/main/linprog/allocation_ampl.ipynb)

In [ ]:
%pip install amplpy ampl-module-base ampl-module-highs --extra-index-url https://pypi.ampl.com

In [ ]:
import pandas as pd
import requests
from amplpy import ampl_notebook
from google.colab import userdata

ampl = ampl_notebook(license_uuid=userdata.get("AMPL_CE_LICENSE"))

## Explanation

We use the [`amplpy`](https://amplpy.ampl.com/en/latest/) Jupyter integration
to specify the model in a cell rather than a separate file.  This helps us
see the entire picture at a glance.  The model also uses several advanced
[AMPL](https://dev.ampl.com/) features to keep its description terse:

- [Compound Sets](https://ampl.com/wp-content/uploads/Chapter-6-Compound-Sets-and-Indexing-AMPL-Book.pdf)
to associate stocks with sectors.  The approach comes from a 
[Google Group answer](https://groups.google.com/g/ampl/c/ULGck_3EQOM/m/yHvDLokzBQAJ);
- [Columnwise Formulation](https://ampl.com/wp-content/uploads/Chapter-16-Columnwise-Formulations-AMPL-Book.pdf)
to minimize repetition.  Look for `to_come`, `obj`, and `coeff`;
- [Piecewise-Linear functions](https://ampl.com/wp-content/uploads/Chapter-17-Piecewise-Linear-Programs-AMPL-Book.pdf)
to define penalty terms.  The AMPL book explains that in "model declarations,
no variables may appear in the _breakpoint-list_, _slope-list_ and 
_zero-expr_ (if any), while an _arg-expr_ can only be a reference to an
individual variable".  These restrictions explain in part why we
introduce more helper variable than appears strictly necessary.  Note
how flexibly [`pandas`](https://pandas.pydata.org/) 
[`MultiIndex`](https://pandas.pydata.org/docs/user_guide/advanced.html)
lets us parametrize penalty terms.

### Choice of representation

The primary decision variable `X` is defined over the Cartesian
product `STOCKS` ⨯ `ACCOUNTS` under the assumption that each
account accepts all stocks.  Its bounds imply that any account
can take the entire holding in a particular stock while no
account can hold a position with a different sign from
other accounts.  This presents two immediate benefits:

1. Gross market values only require a simple multiplication rather then
helper variables;
1. Boxes (where a stock is short in one account and long in another)
are impossible by construction.

In [ ]:
%%ampl_eval
set STOCKS;
set ACCOUNTS = {1..2};
set SECTORS;
# see https://groups.google.com/g/ampl/c/ULGck_3EQOM/m/yHvDLokzBQAJ
set STOCKS_IN_SECTOR {SECTORS} within STOCKS;
set PENALTIES = {"concentration", "sector"};

param base {ACCOUNTS};
param market_value {STOCKS};
param sign {s in STOCKS} = if market_value[s] >= 0 then 1 else -1;
param volume {STOCKS};
param sizes {PENALTIES, ACCOUNTS} default 0 >= 0;
set ACCOUNTS_WITH_CONC = {a in ACCOUNTS: sizes["concentration", a]>0};
set ACCOUNTS_WITH_SECT = {a in ACCOUNTS: sizes["sector", a]>0};
param breaks {p in PENALTIES, a in ACCOUNTS, i in 1..sizes[p, a]}
          >= if i = 1 then 0.0 else breaks[p, a, i-1];
param slopes {p in PENALTIES, a in ACCOUNTS, i in 1..sizes[p, a]}
          > if i = 1 then 0.0 else slopes[p, a, i-1];

minimize TotalCost;

subject to Completeness {s in STOCKS}:
          to_come = market_value[s];

var X {s in STOCKS, ACCOUNTS}
          >= <<0; 1,0>> market_value[s], <= <<0; 0,1>> market_value[s],
          coeff Completeness[s] 1;

subject to AbsXLevel {s in STOCKS, a in ACCOUNTS}:
          to_come = sign[s] * X[s, a];

var AbsX {s in STOCKS, a in ACCOUNTS} >= 0,
          obj TotalCost base[a],
          coeff AbsXLevel[s, a] 1;

subject to GMVLevel {a in ACCOUNTS}:
          to_come = sum {s in STOCKS} AbsX[s, a];

var GMV {a in ACCOUNTS} >= 0,
          coeff GMVLevel[a] 1;

subject to SectorMVLevel {sector in SECTORS, a in ACCOUNTS_WITH_SECT}:
          to_come = sum {s in STOCKS_IN_SECTOR[sector]} X[s, a];

var SectorMV {sector in SECTORS, a in ACCOUNTS_WITH_SECT},
          coeff SectorMVLevel[sector, a] 1;

subject to ConcentrationPenaltyLevel {s in STOCKS, a in ACCOUNTS_WITH_CONC, p in 1..sizes["concentration", a]}:
          to_come >= AbsX[s, a] - breaks["concentration", a, p] * GMV[a];

var ConcentrationPenalty {s in STOCKS, a in ACCOUNTS_WITH_CONC, p in 1..sizes["concentration", a]} >= 0,
          obj TotalCost if p = 1 then slopes["concentration", a, p] else slopes["concentration", a, p] - slopes["concentration", a, p-1],
          coeff ConcentrationPenaltyLevel[s, a, p] 1;

subject to SectorPenaltyLevel {sector in SECTORS, a in ACCOUNTS_WITH_SECT, p in 1..sizes["sector", a]}:
          to_come >= <<0; -1,1>> SectorMV[sector, a] - breaks["sector", a, p] * GMV[a];

var SectorPenalty {sector in SECTORS, a in ACCOUNTS_WITH_SECT, p in 1..sizes["sector", a]} >= 0,
          obj TotalCost if p = 1 then slopes["sector", a, p] else slopes["sector", a, p] - slopes["sector", a, p-1],
          coeff SectorPenaltyLevel[sector, a, p] 1;

In [ ]:
response = requests.get(
    "https://www.harborcapital.com/page-data/etf/lseq/page-data.json"
)
response.raise_for_status()
tabs = response.json()["result"]["data"]["contentstackProductV2"]["product_tabs"]
full_holdings = next(
    section for tab in tabs
    if (data_section := tab["data_section"])
    and data_section["title"] == "Holdings"
    for section in data_section["section"]
    if section["title"] == "Full Holdings"
)
holdings = (
    pd.json_normalize(full_holdings, ["api_reference", "data", "fullHoldings"])
    .set_index("ticker")[["marketValue"]]
    .rename(columns={"marketValue": "market_value"})
)

response = requests.get(
    "https://api.nasdaq.com/api/screener/stocks",
    params={
        "tableonly": False,
        "limit": 10_000,
        "download": True,
    },
    headers={"User-Agent": ""}
)
response.raise_for_status()
data = response.json()

screener = (
    pd.json_normalize(data, ["data", "rows"])
    .rename(columns=data["data"]["headers"])
    .set_index("Symbol")[["Volume", "Sector"]]
    .rename(columns={"Volume": "volume"})
    .astype({"volume": float})
)
π = holdings.join(screener, how="inner")
π

In [ ]:
penalties = pd.DataFrame(
    [["concentration", 1, 1, 0.1, 0.1], ["sector", 1, 1, 0.2, 0.1]],
    columns=["name", "account", "index", "breaks", "slopes"],
).set_index(["name", "account", "index"])
penalties

In [ ]:
ampl.set_data(π[["market_value", "volume"]], set_name="STOCKS")
ampl.param["base"] = {1: 0.05, 2: 0.06}
ampl.param["sizes"] = penalties["breaks"].groupby(["name", "account"]).count()
ampl.set_data(penalties)

stocks_in_sector = π.groupby("Sector").groups
ampl.set["SECTORS"] = stocks_in_sector.keys()
ampl.set["STOCKS_IN_SECTOR"] = stocks_in_sector

ampl.option["solver"] = "highs"
ampl.option["highs_options"] = {"outlev": 1}
ampl.solve()
assert ampl.solve_result == "solved"

In [ ]:
X = ampl.get_variable("X").get_values().to_pandas()
π.assign(
    account_1=X.loc[(π.index, 1), "X.val"].to_numpy(),
    account_2=X.loc[(π.index, 2), "X.val"].to_numpy(),
)